# Find Bell Inequality
> Outputs a random Bell inequality for a given scenario and tries to find a quantum violation for it.<br></br>
> Currently limited to 3 parties and input signaling only

---

## Preamble

In [440]:
from ncpol2sdpa import *
import numpy as np
import itertools as it
import time
import cvxpy as cvx


---
## Scenario definition

In [490]:
def genSeq(num_list_in):
    """Generates the sequence itertools.product(range(num_list_in[0]),range(num_list_in[1]),...)"""
    return it.product(*[range(n) for n in num_list_in])
    
def join_val(algs, X):
    
    X = np.cumprod([1]+list(X)[:-1])
    return np.sum(np.array(algs)*X)

##################################################################################################
# Setting up the scenario

q_inputs = q_in_Alice, q_in_Bob, q_in_Charlie = 2,2,2
q_outs = q_out_Alice, q_out_Bob, q_out_Charlie = 2,2,2

TotA, TotX = np.prod(q_outs), np.prod(q_inputs)

q_parties = 3

# Signaling structure of the scenario
#
# Considering only inputs to outputs. Each list is 
# a child node and numbers correspond to the pa-
# rents that influence each child.
# Order matters for the ReducedProb function
signaling_struct = [[0],[1],[2,1,0]]

##################################################
# Useful functions
##################################################
def scn_input(party,*ins):
    child = signaling_struct[party]
    
    sz_in = [q_inputs[c] for c in child]
    
    if len(ins) == q_parties:
        return join_val([ins[c] for c in child],sz_in)
    elif len(ins) == len(child):
        return join_val(ins,sz_in)
    
    return -1

##################################################
# Extended set of inputs to accommodate the 
# signaling structure of the scenario
# Parties are ordered as A,B,C
##################################################
def get_full_inputs():
    tmp_in = np.array(q_inputs)
    return tuple([np.prod(tmp_in[c]) for c in signaling_struct])

q_inputs_ext = get_full_inputs()

##################################################
# Full set of probabilities for this scenario
##################################################
p_list = [ [q_outs[j]]*q_inputs_ext[j] for j in range(q_parties)]
ProbFull = Probability(*p_list)

##################################################
# Calling the reduced form of the probability
##################################################

def ReducedProb(outs, parties=['Alice','Bob','Charlie'],x=0,y=0,z=0):
        
    party_idx = {'Alice':0, 'Bob':1, 'Charlie':2}
    ncpol_id = ['A','B','C']

    p_seq = list(set([party_idx[p] for p in parties]))
    p_seq.sort()

    full_input = [scn_input(i,x,y,z) for i in range(q_parties)]

    return ProbFull(outs,[full_input[p] for p in p_seq], [ncpol_id[p] for p in p_seq])

##################################################
# Objective function and constraints
##################################################

def get_objective(Bell):

    eq_viol = 0
    for a,b,c,x,y,z in genSeq(q_outs+q_inputs):
        eq_viol -= Bell[a,b,c,x,y,z]*ReducedProb([a,b,c], parties=['Alice','Bob','Charlie'], x=x,y=y,z=z)

    return eq_viol

def get_trivial_ineqs():
    # Nonnegativity inequalities for the distributions
    ineq_triv = []
    for a,b,c,x,y,z in genSeq(q_outs + q_inputs_ext):
        ineq_triv += [ProbFull([a,b,c],[x,y,z])]
    return ineq_triv
        
##################################################
# Information retrieval
##################################################

AllOps  = ProbFull.get_all_operators()
AllSubs = ProbFull.substitutions

def get_extra_monomials(extra_monomials):
    return ProbFull.get_extra_monomials(*extra_monomials)


##################################################
# Vertices of the scenario
##################################################

def get_vertices():
    # Generating the vertices of the current scenario
    
    # Number of deterministic behaviors for each party
    Lamb = [q_outs[i]**q_inputs_ext[i] for i in range(q_parties)]
    q_vtx = np.prod(Lamb)
    
    Vtx = np.zeros((TotA,TotX,q_vtx))
    
    # Deterministic responses
    A = [np.zeros((q_inputs_ext[i],Lamb[i]),dtype=np.int32) for i in range(q_parties)]
    
    for i in range(q_parties):
        for lam in range(Lamb[i]):
            L = lam
            for x in range(q_inputs_ext[i]):
                A[i][x,lam] = L%q_outs[i]
                L = L//q_outs[i]
    
    
    lm = np.zeros((q_parties,),dtype=np.int32)
    for lam in range(q_vtx):
        lm_b = lam
        for i in range(q_parties):
            lm[i] = lm_b%Lamb[i]
            lm_b = lm_b//Lamb[i]
                    
        for x,y,z in genSeq(q_inputs):
            out = join_val([A[i][scn_input(i,x,y,z),lm[i]] for i in range(q_parties)],q_outs)
            inp = join_val([x,y,z],q_inputs)
            Vtx[out,inp,lam] = 1
            
    return np.reshape(Vtx,(TotA*TotX,q_vtx),order="F")-np.ones((TotA*TotX,q_vtx))/TotA, q_vtx
                

In [491]:
VtxMat, q_vtx = get_vertices()

---
## Generating a random inequality and finding its maximal Quantum Violation

In [492]:
##################################################
# Parameters for the search and initial values
##################################################
max_time = 30*60
max_attempts = 100
eps = 10**-5
log_pt = 10

##################################################
# Parameters for the SDP
##################################################
NPA_level = 2
verbose = True
c_solver = 'mosek'
extra_monomials = None #['ABC'] #['ABC','AAA','BBB','CCC']

##################################################
# Getting the NPA relaxation
##################################################
sdp_guess = SdpRelaxation(AllOps, verbose=verbose, parallel=True)

ineq_triv  = get_trivial_ineqs()
if extra_monomials==None:
    tic = time.perf_counter()
    sdp_guess.get_relaxation(NPA_level, objective=0,\
                             inequalities=ineq_triv,\
                             substitutions=AllSubs)
    tac = time.perf_counter()

else:
    tic = time.perf_counter()
    sdp_guess.get_relaxation(NPA_level, objective=0,\
                             inequalities=ineq_triv,\
                             substitutions=AllSubs,\
                             extramonomials=get_extra_monomials(extra_monomials))
    tac = time.perf_counter()

relax_time = tac-tic

##################################################
# Auxiliary variables for the random point 
# generation
##################################################

# Radius of the local polytope
LocRad = np.sqrt(TotX*(1 + 1/TotA))

# White noise
Noise = np.ones(q_outs+q_inputs)/TotA

# Valid points must satisfy normalization and NS constraints
# Normalization comes entirely from the Noise part, the ran-
# dom point should satisfy a "0-sum" condition, thus being
# orthogonal to the normalized-vectors subspace. It should, 
# equivalently, belong to the kernel of a matrix with all
# constraints

CMat = []

# Alice NS constraint
for b,y,z in genSeq([q_outs[1],q_inputs[1],q_inputs[2]]):
    Tmp_mat = np.zeros(q_outs + q_inputs)
    Tmp_mat[:,b,:,0,y,z] = np.ones((q_outs[0],q_outs[2]))
    Tmp_mat[:,b,:,1,y,z] = -np.ones((q_outs[0],q_outs[2]))
    
    Tmp_mat = np.reshape(Tmp_mat,(-1,),order="F")
    CMat += [Tmp_mat/np.linalg.norm(Tmp_mat)]

# Bob NS
for a,x,z in genSeq([q_outs[0],q_inputs[0],q_inputs[2]]):
    Tmp_mat = np.zeros(q_outs + q_inputs)
    Tmp_mat[a,:,:,x,0,z] = np.ones((q_outs[1],q_outs[2]))
    Tmp_mat[a,:,:,x,1,z] = -np.ones((q_outs[1],q_outs[2]))
    
    Tmp_mat = np.reshape(Tmp_mat,(-1,),order="F")
    CMat += [Tmp_mat/np.linalg.norm(Tmp_mat)]

# Charlie NS
for a,b,x,y,z in genSeq([q_outs[0], q_outs[1], q_inputs[0], q_inputs[1], q_inputs[2]-1]):
    Tmp_mat = np.zeros(q_outs + q_inputs)
    Tmp_mat[a,b,:,x,y,0] =  1
    Tmp_mat[a,b,:,x,y,z+1] = -1

    Tmp_mat = np.reshape(Tmp_mat,(-1,),order="F")
    CMat += [Tmp_mat/np.linalg.norm(Tmp_mat)]

# 0 sum constraint
ProjOne = np.kron(np.eye(TotX),np.ones((1,TotA)))/np.sqrt(TotA)

# Constraint matrix
CMat = np.vstack([np.array(CMat),ProjOne])

# We will randomize a point in the NS subspace, 0-sum subspace
# Note that all projectors commute two-by-two
tmp_eval,ns_basis = np.linalg.eigh(CMat.T@CMat)
ns_basis = ns_basis[:,abs(tmp_eval) < 0.1]
q_ns_vec = ns_basis.shape[1]

# Variables of the LP to obtain the inequality
Ineq = cvx.Variable(TotA*TotX)
Phi = cvx.Variable()
Constraints = [ Phi*np.ones((q_vtx,)) >= Ineq.T @ VtxMat, Phi <= 1]

##################################################
# Search for a good inequality
##################################################

# Initializing control variables
Found = False
Timeout = False
FirstRun = True
q_attempts = 0
prev_pt = 0
Elapsed = 0

# Main loop
while not (Found or Timeout) and q_attempts < max_attempts:
    t0 = time.monotonic()
    
    # Generating a random point and a random inequality

    PtRnd = ns_basis@np.random.normal(size=(q_ns_vec,))
    
    # We don't need to add noise here, since we remove for the LP below
    PtRnd = LocRad*PtRnd/np.linalg.norm(PtRnd)
    
    ##################################################
    # Getting the inequality to the random point
    # -- Ensure that VtxMat is loaded in the preamble
    ##################################################
    
    CvxProblem = cvx.Problem(cvx.Maximize(Ineq.T@PtRnd - Phi), Constraints)
    CvxProblem.solve(cvx.MOSEK, warm_start=False)

    Bound = np.max(Ineq.value.T@(VtxMat + np.ones((TotA*TotX,q_vtx))/TotA))

    Ineq_Coef = np.reshape(Ineq.value,q_outs+q_inputs,order="F")

    ##################################################################################################
    # SDP BLOCK
    

    ##################################################
    # Updating the objective and solving the SDP
    ##################################################
    sdp_guess.set_objective(get_objective(Ineq_Coef))
    
    tic = time.perf_counter()
    sdp_guess.solve(c_solver)
    tac = time.perf_counter()

    solve_time = tac-tic

    ##################################################
    # Output
    ##################################################
    
    if q_attempts - prev_pt == log_pt:
        cur_elapse = Elapsed + time.monotonic() - t0
        print('Attempt no. {}'.format(q_attempts))
        print('Time elapsed: {:.0f} min {:.2f} s'.format(cur_elapse//60,cur_elapse%60))
        print('Current values:')
        print('\tBound: {:.3f}'.format(Bound))
        print('\tOptimal value: {:.3f}'.format(-sdp_guess.primal)) 
        print()
        prev_pt = q_attempts
              
    if not sdp_guess.status == 'optimal':
        print('Solver status {} at attempt no. {}').format(sdp_guess.status, q_attempts)
        print('Current values:')
        print('\tBound: {:.3f}'.format(Bound))
        print('\tOptimal value: {:.3f}'.format(-sdp_guess.primal)) 
        print()
    
    if abs(-sdp_guess.primal-Bound) > eps:
        Found = True
        print('Found a good inequality -- Bound: {:.3f}, Violation: {:.3f}'.format(Bound, -sdp_guess.primal))
        
    q_attempts += 1
    
    Elapsed += time.monotonic() - t0
    if Elapsed > max_time:
        Timeout = True
        print('Timed out -- Elapsed time: {:.0f} min {:.2f} s'.format(np.floor(Elapsed//60),Elapsed%60))
              

The problem has 12 noncommuting Hermitian variables
Parallel processing on 4 cores
Calculating block structure...
Estimated number of SDP variables: 5994
Generating moment matrix...
Reduced number of SDP variables: 4298298 (done: 100.02%, ETA 00:00:-0.0)
Processing 256/256 constraints...
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1      

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 66795           
Factor     - setup time             : 1.36              dense det. time        : 0.00            
Factor     - ML order time          : 0.94              GP order time          : 0.00            
Factor     - nonzeros before factor 

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 66795           
Factor     - setup time             : 1.31              dense det. time        : 0.00            
Factor     - ML order time          : 0.94              GP order time          : 0.00            
Factor     - nonzeros before factor 

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 66795           
Factor     - setup time             : 1.22              dense det. time        : 0.00            
Factor     - ML order time          : 0.86              GP order time          : 0.00            
Factor     - nonzeros before factor 

...

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 66795           
Factor     - setup time             : 1.20              dense det. time        : 0.00            
Factor     - ML order time          : 0.84              GP order time          : 0.00            
Factor     - nonzeros before factor 

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 66795           
Factor     - setup time             : 1.20              dense det. time        : 0.00            
Factor     - ML order time          : 0.86              GP order time          : 0.00            
Factor     - nonzeros before factor 

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 66795           
Factor     - setup time             : 1.19              dense det. time        : 0.00            
Factor     - ML order time          : 0.84              GP order time          : 0.00            
Factor     - nonzeros before factor 

Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 4298            
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 4298
Optimizer  - Cones                  : 0
Optimizer  - Scalar variables       : 0                 conic                  : 0               
Optimizer  - Semi-definite variables: 1                 scalarized             : 66795           
Factor     - setup time             : 1.16              dense det. time        : 0.00            
Factor     - ML order time          : 0.81              GP order time          : 0.00            
Factor     - nonzeros before factor 